In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
chunk_size = 10000  # Size of each chunk
chunks = []  # This list will hold the chunks of DataFrame.

# Process the file in chunks
for chunk in pd.read_json('/content/drive/MyDrive/your-unzip-folder/goodreads_reviews_dedup.json', lines=True, chunksize=chunk_size):
    chunks.append(chunk)

# Combine chunks back into a single DataFrame
df_non_spoiler = pd.concat(chunks, ignore_index=True)

OSError: ignored

In [ ]:
df_spoiler = pd.read_json('/content/drive/MyDrive/your-unzip-folder/goodreads_reviews_spoiler.json', lines=True)
df_spoiler.head()

,user_id,timestamp,review_sentences,rating,has_spoiler,book_id,review_id
0,8842281e1d1347389f2ab93d60773d4d,2017-08-30,"[[0, This is a special book.], [0, It started ...",5,True,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb
1,8842281e1d1347389f2ab93d60773d4d,2017-03-22,"[[0, Recommended by Don Katz.], [0, Avail for ...",3,False,16981,a5d2c3628987712d0e05c4f90798eb67
2,8842281e1d1347389f2ab93d60773d4d,2017-03-20,"[[0, A fun, fast paced science fiction thrille...",3,True,28684704,2ede853b14dc4583f96cf5d120af636f
3,8842281e1d1347389f2ab93d60773d4d,2016-11-09,"[[0, Recommended reading to understand what is...",0,False,27161156,ced5675e55cd9d38a524743f5c40996e
4,8842281e1d1347389f2ab93d60773d4d,2016-04-25,"[[0, I really enjoyed this book, and there is ...",4,True,25884323,332732725863131279a8e345b63ac33e


In [ ]:
# Select only 'review_id' and 'has_spoiler' from df_spoiler
df_spoiler = df_spoiler[['review_id', 'has_spoiler']]

# Ensure both DataFrames have 'review_id' as the same data type
df_non_spoiler['review_id'] = df_non_spoiler['review_id'].astype(str)
df_spoiler['review_id'] = df_spoiler['review_id'].astype(str)

# Merge the two DataFrames on 'review_id' using an inner join
df_merged = pd.merge(df_non_spoiler, df_spoiler, on='review_id', how='inner')

# Save the merged DataFrame
output_path = '/content/drive/MyDrive/your-unzip-folder/merged_dataframe.csv'
df_merged.to_csv(output_path, index=False)

In [ ]:
df_merged.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,has_spoiler
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1,True
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,,,1,0,False
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0,True
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,,,5,1,False
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1,True


In [ ]:
# Load the merged DataFrame
df_merged = pd.read_csv('/content/drive/MyDrive/your-unzip-folder/merged_dataframe.csv')

# Read the list of book_ids from the text file
with open('/content/drive/MyDrive/your-unzip-folder/books_over_200_reviews.txt', 'r') as file:
    book_ids = file.read().splitlines()

# Convert book_ids to the correct data type (the same as in df_merged)
book_ids = [str(id) for id in book_ids]

# Filter the DataFrame
df_filtered = df_merged[df_merged['book_id'].isin(book_ids)]

# Optionally, save the filtered DataFrame
output_path_filtered = '/content/drive/MyDrive/your-unzip-folder/filtered_dataframe.csv'
df_filtered.to_csv(output_path_filtered, index=False)

In [ ]:
df_filtered.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,has_spoiler


Start running from here. Above code is to load and merge data.

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
chunk_size = 10000  # Size of each chunk
chunks = []  # This list will hold the chunks of DataFrame.

# Process the file in chunks
for chunk in pd.read_csv('/content/drive/MyDrive/your-unzip-folder/merged_dataframe.csv', chunksize=chunk_size):
    chunks.append(chunk)

# Combine chunks back into a single DataFrame
df_merged = pd.concat(chunks, ignore_index=True)

In [ ]:
df_merged.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,has_spoiler
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1,True
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0,False
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0,True
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,NaN,NaN,5,1,False
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1,True


In [ ]:
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords

# Ensure the stopwords are downloaded
nltk.download('stopwords')

# Get a set of English stop words
stop_words = set(stopwords.words('english'))

# Function to find most common words excluding stop words
def most_common_words(text):
    words = [word for word in str(text).lower().split() if word.isalpha() and word not in stop_words]
    return Counter(words).most_common(1)[0][0] if words else None

# Process in chunks
for start_row in range(0, df_merged.shape[0], chunk_size):
    end_row = min(start_row + chunk_size, df_merged.shape[0])
    df_merged.loc[start_row:end_row, 'most_common_words'] = df_merged['review_text'][start_row:end_row].apply(most_common_words)

print(df_merged['most_common_words'].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0           book
1    recommended
2           book
3    recommended
4        meaning
Name: most_common_words, dtype: object


In [ ]:
for index, row in df_merged.iterrows():
    print(f"Row {index}: {row['most_common_words']}")

Streaming output truncated to the last 5000 lines.
Row 1373033: make
Row 1373034: book
Row 1373035: trudged
Row 1373036: book
Row 1373037: great
Row 1373038: think
Row 1373039: thought
Row 1373040: book
Row 1373041: book
Row 1373042: book
Row 1373043: toward
Row 1373044: book
Row 1373045: took
Row 1373046: relate
Row 1373047: book
Row 1373048: much
Row 1373049: ummmm
Row 1373050: love
Row 1373051: really
Row 1373052: writing
Row 1373053: loved
Row 1373054: book
Row 1373055: say
Row 1373056: great
Row 1373057: great
Row 1373058: soooo
Row 1373059: get
Row 1373060: words
Row 1373061: book
Row 1373062: annie
Row 1373063: noah
Row 1373064: want
Row 1373065: cathy
Row 1373066: whole
Row 1373067: story
Row 1373068: book
Row 1373069: story
Row 1373070: wants
Row 1373071: life
Row 1373072: loved
Row 1373073: read
Row 1373074: book
Row 1373075: wanted
Row 1373076: loved
Row 1373077: loved
Row 1373078: love
Row 1373079: middle
Row 1373080: leading
Row 1373081: loved
Row 1373082: really
Row 13730

In [ ]:
import pandas as pd
from gensim import corpora, models
import gensim
import nltk
from nltk.corpus import stopwords
import string

# Ensure you have the required NLTK packages
# nltk.download('stopwords')

# Preprocessing function
def preprocess(text):
    # Convert non-string data to string
    text = str(text) if text is not None else ''

    tokens = gensim.utils.simple_preprocess(text)
    stop_words = stopwords.words('english')
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]
    return tokens

# Initialize empty dictionary and empty corpus
dictionary = corpora.Dictionary()
corpus = []
num_topics = 5

# Process in chunks
for start_row in range(0, df_merged.shape[0], chunk_size):
    end_row = min(start_row + chunk_size, df_merged.shape[0])
    chunk = df_merged['review_text'][start_row:end_row].map(preprocess)

    # Update the dictionary and extend the corpus
    dictionary.add_documents(chunk)
    corpus.extend([dictionary.doc2bow(text) for text in chunk])

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=10, workers=2)

# Print the Keyword in the topics
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.013*"love" + 0.009*"story" + 0.008*"one" + 0.006*"life" + 0.006*"loved" + 0.005*"romance" + 0.005*"relationship" + 0.005*"get" + 0.004*"family" + 0.004*"two"
Topic: 1 
Words: 0.038*"book" + 0.019*"read" + 0.018*"story" + 0.018*"really" + 0.015*"characters" + 0.014*"like" + 0.010*"one" + 0.008*"much" + 0.008*"good" + 0.008*"character"
Topic: 2 
Words: 0.038*"book" + 0.019*"like" + 0.015*"read" + 0.013*"really" + 0.012*"one" + 0.012*"love" + 0.010*"know" + 0.008*"even" + 0.008*"much" + 0.008*"would"
Topic: 3 
Words: 0.006*"one" + 0.006*"life" + 0.006*"people" + 0.005*"story" + 0.004*"novel" + 0.004*"book" + 0.004*"time" + 0.004*"world" + 0.004*"would" + 0.003*"like"
Topic: 4 
Words: 0.018*"series" + 0.012*"world" + 0.008*"one" + 0.007*"book" + 0.006*"characters" + 0.006*"new" + 0.005*"love" + 0.005*"first" + 0.005*"magic" + 0.005*"fantasy"


In [ ]:
import pandas as pd
from gensim import corpora, models
import gensim
import nltk
from nltk.corpus import stopwords
import string

nltk.download('stopwords')

# Preprocessing function
def preprocess(text):
    # Convert non-string data to string
    text = str(text) if text is not None else ''

    tokens = gensim.utils.simple_preprocess(text)
    stop_words = stopwords.words('english')
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]
    return tokens

# Initialize empty dictionary and empty corpus
dictionary = corpora.Dictionary()
corpus = []
num_topics = 5

# Process in chunks
for start_row in range(0, df_non_spoiler.shape[0], chunk_size):
    end_row = min(start_row + chunk_size, df_non_spoiler.shape[0])
    chunk = df_non_spoiler['review_text'][start_row:end_row].map(preprocess)

    # Update the dictionary and extend the corpus
    dictionary.add_documents(chunk)
    corpus.extend([dictionary.doc2bow(text) for text in chunk])

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=10, workers=2)

# Print the Keyword in the topics
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))